In [ ]:
!pip install numpy-stl


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile


zip_file_path = "/content/drive/MyDrive/datasets/stl_1000-20240321T094418Z-001.zip"


extract_dir = "/content/PDSVision_dataset/PDSVision_dataset"


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


In [ ]:
pip install numpy trimesh torch scikit-learn matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
# 1
import os
import trimesh
import numpy as np
def stl_to_voxel(file_path, resolution=32, max_size=(32, 32, 32)):
    """Converts an STL file to a uniformly sized voxel grid."""
    mesh = trimesh.load(file_path)
    scale_factor = max(mesh.extents) / resolution
    voxel_grid = mesh.voxelized(pitch=scale_factor)

    # Convert voxel grid to boolean array
    voxel_matrix = voxel_grid.matrix

    # Pad or trim the voxel matrix to fit max_size
    padded_voxel_matrix = np.zeros(max_size, dtype=np.float32)  # Initialize a zero array of max_size
    # Calculate slicing indexes
    slices = tuple(slice(0, min(max_size[i], voxel_matrix.shape[i])) for i in range(3))
    padded_voxel_matrix[slices] = voxel_matrix[slices]

    return padded_voxel_matrix

# Directory containing STL files
stl_directory = '/content/PDSVision_dataset/PDSVision_dataset/stl_1000'

voxel_data = []

file_names = []
for filename in os.listdir(stl_directory):
    file_path = os.path.join(stl_directory, filename)
    if filename.endswith('.stl'):
        try:
            voxel_grid = stl_to_voxel(file_path)
            voxel_data.append(voxel_grid)
            file_names.append(filename)
        except Exception as e:
            print(f"Failed to process {filename}: {str(e)}")

# Now, converting the list of voxel grids into a numpy array should work
voxel_data = np.stack(voxel_data)
print("Voxel data shape:", voxel_data.shape)

Voxel data shape: (1000, 32, 32, 32)


In [ ]:
# 2
import torch
import torch.nn as nn
import torch.optim as optim

# Assuming voxel_data is ready and properly shaped
voxel_tensors = torch.tensor(voxel_data, dtype=torch.float).unsqueeze(1)  # Add channel dimension

class VoxelCNN(nn.Module):
    def __init__(self):
        super(VoxelCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2),
            nn.Conv3d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2),
            nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2)
        )
        self.adaptive_pool = nn.AdaptiveAvgPool3d((1, 1, 1))  # Adaptively pools to (1, 1, 1)
        self.classifier = nn.Linear(64, 10)

    def forward(self, x):
        x = self.features(x)
        x = self.adaptive_pool(x)  # Ensures output is always (batch_size, 64, 1, 1, 1)
        x = torch.flatten(x, 1)  # Flatten the dimensions
        x = self.classifier(x)
        return x
# Expected Output:
# A neural network model capable of processing 3D voxel grids and producing a classification output of 10 classes.

model = VoxelCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# expected output:A neural network model ready for training with a specified loss function and optimizer.

# Dummy target for demonstration
targets = torch.randint(0, 10, (len(voxel_tensors),))
# exp output:A tensor of shape (N,) where N is the number of voxel grids, containing random integer targets for demonstration
# Training loop (simplified)
for epoch in range(50):  # Loop over the dataset multiple times
    optimizer.zero_grad()
    outputs = model(voxel_tensors)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    print('Epoch:', epoch+1, 'Loss:', loss.item())

# exp output:The loss value printed for each epoch, showing how the loss decreases (or fluctuates) over 50 epochs.    


Epoch: 1 Loss: 2.3106751441955566
Epoch: 2 Loss: 2.308455467224121
Epoch: 3 Loss: 2.3065176010131836
Epoch: 4 Loss: 2.304062604904175
Epoch: 5 Loss: 2.3011655807495117
Epoch: 6 Loss: 2.298424482345581
Epoch: 7 Loss: 2.296672821044922
Epoch: 8 Loss: 2.296318292617798
Epoch: 9 Loss: 2.2963058948516846
Epoch: 10 Loss: 2.295654773712158
Epoch: 11 Loss: 2.2945716381073
Epoch: 12 Loss: 2.2934658527374268
Epoch: 13 Loss: 2.2925055027008057
Epoch: 14 Loss: 2.2917003631591797
Epoch: 15 Loss: 2.29095721244812
Epoch: 16 Loss: 2.2901456356048584
Epoch: 17 Loss: 2.2892017364501953
Epoch: 18 Loss: 2.2880918979644775
Epoch: 19 Loss: 2.2867932319641113
Epoch: 20 Loss: 2.285322904586792
Epoch: 21 Loss: 2.2837183475494385
Epoch: 22 Loss: 2.2820041179656982
Epoch: 23 Loss: 2.2802608013153076
Epoch: 24 Loss: 2.2785210609436035
Epoch: 25 Loss: 2.2766613960266113
Epoch: 26 Loss: 2.2746405601501465
Epoch: 27 Loss: 2.2724969387054443
Epoch: 28 Loss: 2.2703404426574707
Epoch: 29 Loss: 2.2683234214782715
Epoch:

In [ ]:
from stl_metadata import Surface,Facet

In [ ]:
import pickle


In [ ]:
# 3
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil

def calculate_centroid(mesh_data):
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    G = nx.Graph()

    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)

# calculate_centroid: A 1D numpy array of three elements representing the centroid coordinates of the mesh.
# extract_holes_and_cavities: An integer representing the number of holes or cavities in the mesh.
def collect_features(folder_path, model, voxel_tensors):
    features_list = []
    stl_files = [f for f in os.listdir(folder_path) if f.endswith('.stl')]

    # Ensure the model is in evaluation mode
    model.eval()

    # Extract features from voxel data using the model
    with torch.no_grad():
        extracted_features = model.features(voxel_tensors).mean([2, 3, 4]).cpu().numpy()

    for i, stl_file in enumerate(stl_files):
        stl_file_path = os.path.join(folder_path, stl_file)
        mesh_data = mesh.Mesh.from_file(stl_file_path)

        centroid = calculate_centroid(mesh_data)
        cavities = extract_holes_and_cavities(mesh_data)

        surface = Surface()
        surface.load(stl_file_path)
        surface_area = surface.get_area()
        bbv = surface.get_bounding_box_volume()

        # Concatenate geometric features with features from the voxel CNN
        combined_features = np.concatenate([
            np.array(centroid),
            np.array([surface_area]),
            np.array([bbv]),
            extracted_features[i]  # Assuming alignment of features and files
        ])

        features_list.append(combined_features)

    features_array = np.array(features_list)
    imputer = SimpleImputer(strategy='mean')
    features_array_imputed = imputer.fit_transform(features_array)

    return features_array_imputed, stl_files
# 
def find_optimal_clustering(features, max_files_per_cluster):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    for n_clusters in range(1, len(features) // max_files_per_cluster + 1):
        gmm = GaussianMixture(n_components=n_clusters)
        labels = gmm.fit_predict(X_scaled)
        max_cluster_size = max(np.bincount(labels))
        if max_cluster_size <= max_files_per_cluster:
            return labels, gmm
    return labels, gmm  # In case no configuration satisfies the condition, return last attempt
# exp output: labels: An array of cluster labels assigned to each feature vector.
#             gmm: The fitted Gaussian Mixture Model.
# Scales the features, applies Gaussian Mixture Model clustering, and returns the clustering labels and the GMM. It ensures no cluster exceeds a specified size.
def create_cluster_folders(base_folder, labels, stl_files, folder_path):
    os.makedirs(base_folder, exist_ok=True)
    for cluster_label in np.unique(labels):
        cluster_folder = os.path.join(base_folder, f'Cluster_{cluster_label}')
        os.makedirs(cluster_folder, exist_ok=True)
        files_in_cluster = [stl_files[i] for i in range(len(stl_files)) if labels[i] == cluster_label]
        for stl_file in files_in_cluster:
            source_path = os.path.join(folder_path, stl_file)
            destination_path = os.path.join(cluster_folder, stl_file)
            shutil.copy(source_path, destination_path)
# create_cluster_folders: Creates directories for each cluster and copies the corresponding STL files into these directories.
# Assume 'model' and 'voxel_tensors' are defined
folder_path = "/content/PDSVision_dataset/PDSVision_dataset/stl_1000"
output_folder = "/content/drive/MyDrive/Datasets/encoded_clusters_gmm/"
features, stl_files = collect_features(folder_path, model, voxel_tensors)
labels, gmm = find_optimal_clustering(features, 3)
create_cluster_folders(output_folder, labels, stl_files, folder_path)

In [ ]:
# import os
# import torch
# import shutil
# from sklearn.mixture import GaussianMixture

# # Assuming voxel_tensors, model, and file_names are predefined
# # Also assuming stl_directory is defined where your STL files are located

# # Ensure model is in evaluation mode
# model.eval()

# # Extract features without gradient calculation
# with torch.no_grad():
#     # Assuming voxel_tensors are on the same device as the model
#     extracted_features = model.features(voxel_tensors).mean([2, 3, 4]).cpu().numpy()

# # Use Gaussian Mixture Model for clustering
# gmm = GaussianMixture(n_components=2500)
# clusters = gmm.fit_predict(extracted_features)

# # Save clusters to separate folders
# cluster_directory = '/content/drive/MyDrive/Datasets/encoded_clusters_gmm/'
# os.makedirs(cluster_directory, exist_ok=True)
# for idx, cluster in enumerate(clusters):
#     cluster_folder = os.path.join(cluster_directory, f'cluster_{cluster}')
#     os.makedirs(cluster_folder, exist_ok=True)
#     shutil.copy(os.path.join(stl_directory, file_names[idx]),
#                 os.path.join(cluster_folder, file_names[idx]))



In [ ]:
!pip install hdbscan


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039275 sha256=990562520f91430b10c55e30bea1c86f858c5223478d1c5bcb82746be26241d7
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
Successfully built hdbscan
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [ ]:
# 4
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil

def calculate_centroid(mesh_data):
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    G = nx.Graph()

    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)


def collect_features(folder_path, model, voxel_tensors):
    features_list = []
    stl_files = [f for f in os.listdir(folder_path) if f.endswith('.stl')]

    # Ensure the model is in evaluation mode
    model.eval()

    # Extract features from voxel data using the model
    with torch.no_grad():
        extracted_features = model.features(voxel_tensors).mean([2, 3, 4]).cpu().numpy()

    for i, stl_file in enumerate(stl_files):
        stl_file_path = os.path.join(folder_path, stl_file)
        mesh_data = mesh.Mesh.from_file(stl_file_path)

        centroid = calculate_centroid(mesh_data)
        cavities = extract_holes_and_cavities(mesh_data)

        surface = Surface()
        surface.load(stl_file_path)
        surface_area = surface.get_area()
        bbv = surface.get_bounding_box_volume()

        # Concatenate geometric features with features from the voxel CNN
        combined_features = np.concatenate([
            np.array(centroid),
            np.array([surface_area]),
            np.array([bbv]),
            extracted_features[i]  # Assuming alignment of features and files
        ])

        features_list.append(combined_features)

    features_array = np.array(features_list)
    imputer = SimpleImputer(strategy='mean')
    features_array_imputed = imputer.fit_transform(features_array)

    return features_array_imputed, stl_files
import hdbscan
from sklearn.cluster import KMeans

import hdbscan
from sklearn.preprocessing import StandardScaler

def find_maximum_clusters_hdbscan(features):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)

    # Configure HDBSCAN to produce many small clusters
    clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1,
                                cluster_selection_epsilon=0.1,
                                cluster_selection_method='eom')

    labels = clusterer.fit_predict(X_scaled)

    # Analyze the output
    unique, counts = np.unique(labels, return_counts=True)
    print(f"Total clusters formed (excluding noise): {len(unique[unique != -1])}")
    print(f"Cluster sizes: {dict(zip(unique, counts))}")

    return labels, clusterer
def create_cluster_folders(base_folder, labels, stl_files, folder_path):
    os.makedirs(base_folder, exist_ok=True)
    for cluster_label in np.unique(labels):
        cluster_folder = os.path.join(base_folder, f'Cluster_{cluster_label}')
        os.makedirs(cluster_folder, exist_ok=True)
        files_in_cluster = [stl_files[i] for i in range(len(stl_files)) if labels[i] == cluster_label]
        for stl_file in files_in_cluster:
            source_path = os.path.join(folder_path, stl_file)
            destination_path = os.path.join(cluster_folder, stl_file)
            shutil.copy(source_path, destination_path)
folder_path = "/content/PDSVision_dataset/PDSVision_dataset/stl_1000"
output_folder = "/content/drive/MyDrive/Datasets/encoded_clusters_hdbscan/"
features, stl_files = collect_features(folder_path, model, voxel_tensors)
labels, hdbscan_clusterer = find_maximum_clusters_hdbscan(features)
create_cluster_folders(output_folder, labels, stl_files, folder_path)

# Later in your script:
# labels, hdbscan_clusterer = find_optimal_hdbscan_clustering(features, 3)
# create_cluster_folders(output_folder, labels, stl_files, folder_path)


Total clusters formed (excluding noise): 278
Cluster sizes: {-1: 193, 0: 2, 1: 3, 2: 2, 3: 5, 4: 2, 5: 2, 6: 2, 7: 5, 8: 2, 9: 3, 10: 2, 11: 2, 12: 2, 13: 5, 14: 3, 15: 2, 16: 3, 17: 2, 18: 2, 19: 2, 20: 5, 21: 2, 22: 2, 23: 2, 24: 2, 25: 2, 26: 2, 27: 4, 28: 2, 29: 2, 30: 3, 31: 3, 32: 2, 33: 2, 34: 3, 35: 2, 36: 5, 37: 2, 38: 2, 39: 2, 40: 3, 41: 5, 42: 4, 43: 2, 44: 2, 45: 3, 46: 3, 47: 2, 48: 2, 49: 2, 50: 2, 51: 7, 52: 2, 53: 2, 54: 4, 55: 3, 56: 2, 57: 2, 58: 2, 59: 3, 60: 2, 61: 3, 62: 2, 63: 4, 64: 2, 65: 3, 66: 2, 67: 2, 68: 2, 69: 3, 70: 2, 71: 5, 72: 4, 73: 2, 74: 2, 75: 2, 76: 4, 77: 2, 78: 2, 79: 2, 80: 2, 81: 2, 82: 3, 83: 3, 84: 3, 85: 2, 86: 3, 87: 2, 88: 2, 89: 2, 90: 2, 91: 2, 92: 2, 93: 5, 94: 2, 95: 3, 96: 4, 97: 2, 98: 4, 99: 2, 100: 4, 101: 2, 102: 3, 103: 2, 104: 2, 105: 2, 106: 3, 107: 4, 108: 3, 109: 2, 110: 5, 111: 2, 112: 2, 113: 14, 114: 6, 115: 4, 116: 2, 117: 4, 118: 3, 119: 2, 120: 2, 121: 2, 122: 2, 123: 2, 124: 3, 125: 4, 126: 2, 127: 2, 128: 2, 129: 3,

In [ ]:
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil

def calculate_centroid(mesh_data):
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    G = nx.Graph()

    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)


def collect_features(folder_path, model, voxel_tensors):
    features_list = []
    stl_files = [f for f in os.listdir(folder_path) if f.endswith('.stl')]

    # Ensure the model is in evaluation mode
    model.eval()

    # Extract features from voxel data using the model
    with torch.no_grad():
        extracted_features = model.features(voxel_tensors).mean([2, 3, 4]).cpu().numpy()

    for i, stl_file in enumerate(stl_files):
        stl_file_path = os.path.join(folder_path, stl_file)
        mesh_data = mesh.Mesh.from_file(stl_file_path)

        centroid = calculate_centroid(mesh_data)
        cavities = extract_holes_and_cavities(mesh_data)

        surface = Surface()
        surface.load(stl_file_path)
        surface_area = surface.get_area()
        bbv = surface.get_bounding_box_volume()

        # Concatenate geometric features with features from the voxel CNN
        combined_features = np.concatenate([
            np.array(centroid),

            extracted_features[i]  # Assuming alignment of features and files
        ])

        features_list.append(combined_features)

    features_array = np.array(features_list)
    imputer = SimpleImputer(strategy='mean')
    features_array_imputed = imputer.fit_transform(features_array)

    return features_array_imputed, stl_files
import hdbscan
from sklearn.cluster import KMeans

import hdbscan
from sklearn.preprocessing import StandardScaler

def find_maximum_clusters_hdbscan(features):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)

    # Configure HDBSCAN to produce many small clusters
    clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1,
                                cluster_selection_epsilon=0.1,
                                cluster_selection_method='eom')

    labels = clusterer.fit_predict(X_scaled)

    # Analyze the output
    unique, counts = np.unique(labels, return_counts=True)
    print(f"Total clusters formed (excluding noise): {len(unique[unique != -1])}")
    print(f"Cluster sizes: {dict(zip(unique, counts))}")

    return labels, clusterer
def create_cluster_folders(base_folder, labels, stl_files, folder_path):
    os.makedirs(base_folder, exist_ok=True)
    for cluster_label in np.unique(labels):
        cluster_folder = os.path.join(base_folder, f'Cluster_{cluster_label}')
        os.makedirs(cluster_folder, exist_ok=True)
        files_in_cluster = [stl_files[i] for i in range(len(stl_files)) if labels[i] == cluster_label]
        for stl_file in files_in_cluster:
            source_path = os.path.join(folder_path, stl_file)
            destination_path = os.path.join(cluster_folder, stl_file)
            shutil.copy(source_path, destination_path)
folder_path = "/content/PDSVision_dataset/PDSVision_dataset/stl_1000"
output_folder = "/content/drive/MyDrive/Datasets/encoded_clusters_hdbscan_1/"
features, stl_files = collect_features(folder_path, model, voxel_tensors)
labels, hdbscan_clusterer = find_maximum_clusters_hdbscan(features)
create_cluster_folders(output_folder, labels, stl_files, folder_path)

# Later in your script:
# labels, hdbscan_clusterer = find_optimal_hdbscan_clustering(features, 3)
# create_cluster_folders(output_folder, labels, stl_files, folder_path)


Total clusters formed (excluding noise): 272
Cluster sizes: {-1: 201, 0: 2, 1: 3, 2: 2, 3: 5, 4: 2, 5: 2, 6: 2, 7: 3, 8: 2, 9: 5, 10: 2, 11: 2, 12: 5, 13: 3, 14: 2, 15: 2, 16: 3, 17: 2, 18: 6, 19: 2, 20: 3, 21: 2, 22: 2, 23: 4, 24: 2, 25: 3, 26: 3, 27: 2, 28: 2, 29: 3, 30: 2, 31: 2, 32: 3, 33: 5, 34: 4, 35: 3, 36: 2, 37: 2, 38: 4, 39: 2, 40: 5, 41: 2, 42: 4, 43: 2, 44: 2, 45: 2, 46: 4, 47: 2, 48: 3, 49: 2, 50: 2, 51: 2, 52: 2, 53: 5, 54: 2, 55: 2, 56: 2, 57: 2, 58: 2, 59: 2, 60: 4, 61: 2, 62: 2, 63: 2, 64: 2, 65: 4, 66: 2, 67: 3, 68: 2, 69: 2, 70: 3, 71: 2, 72: 2, 73: 3, 74: 3, 75: 3, 76: 3, 77: 2, 78: 2, 79: 2, 80: 2, 81: 2, 82: 2, 83: 2, 84: 2, 85: 3, 86: 4, 87: 2, 88: 4, 89: 3, 90: 2, 91: 2, 92: 3, 93: 4, 94: 3, 95: 3, 96: 14, 97: 6, 98: 2, 99: 2, 100: 2, 101: 2, 102: 2, 103: 4, 104: 5, 105: 2, 106: 2, 107: 2, 108: 2, 109: 2, 110: 2, 111: 2, 112: 3, 113: 2, 114: 2, 115: 2, 116: 4, 117: 3, 118: 3, 119: 2, 120: 3, 121: 4, 122: 3, 123: 2, 124: 3, 125: 2, 126: 3, 127: 2, 128: 3, 129: 2,

In [ ]:
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil

def calculate_centroid(mesh_data):
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    G = nx.Graph()

    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)


def collect_features(folder_path, model, voxel_tensors):
    features_list = []
    stl_files = [f for f in os.listdir(folder_path) if f.endswith('.stl')]

    # Ensure the model is in evaluation mode
    model.eval()

    # Extract features from voxel data using the model
    with torch.no_grad():
        extracted_features = model.features(voxel_tensors).mean([2, 3, 4]).cpu().numpy()

    for i, stl_file in enumerate(stl_files):
        stl_file_path = os.path.join(folder_path, stl_file)
        mesh_data = mesh.Mesh.from_file(stl_file_path)

        centroid = calculate_centroid(mesh_data)
        cavities = extract_holes_and_cavities(mesh_data)

        surface = Surface()
        surface.load(stl_file_path)
        surface_area = surface.get_area()
        bbv = surface.get_bounding_box_volume()

        # Concatenate geometric features with features from the voxel CNN
        combined_features = np.concatenate([


            extracted_features[i]  # Assuming alignment of features and files
        ])

        features_list.append(combined_features)

    features_array = np.array(features_list)
    imputer = SimpleImputer(strategy='mean')
    features_array_imputed = imputer.fit_transform(features_array)

    return features_array_imputed, stl_files
import hdbscan
from sklearn.cluster import KMeans

import hdbscan
from sklearn.preprocessing import StandardScaler

def find_maximum_clusters_hdbscan(features):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)

    # Configure HDBSCAN to produce many small clusters
    clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1,
                                cluster_selection_epsilon=0.1,
                                cluster_selection_method='eom')

    labels = clusterer.fit_predict(X_scaled)

    # Analyze the output
    unique, counts = np.unique(labels, return_counts=True)
    print(f"Total clusters formed (excluding noise): {len(unique[unique != -1])}")
    print(f"Cluster sizes: {dict(zip(unique, counts))}")

    return labels, clusterer
def create_cluster_folders(base_folder, labels, stl_files, folder_path):
    os.makedirs(base_folder, exist_ok=True)
    for cluster_label in np.unique(labels):
        cluster_folder = os.path.join(base_folder, f'Cluster_{cluster_label}')
        os.makedirs(cluster_folder, exist_ok=True)
        files_in_cluster = [stl_files[i] for i in range(len(stl_files)) if labels[i] == cluster_label]
        for stl_file in files_in_cluster:
            source_path = os.path.join(folder_path, stl_file)
            destination_path = os.path.join(cluster_folder, stl_file)
            shutil.copy(source_path, destination_path)
folder_path = "/content/PDSVision_dataset/PDSVision_dataset/stl_1000"
output_folder = "/content/drive/MyDrive/Datasets/encoded_clusters_hdbscan_2/"
features, stl_files = collect_features(folder_path, model, voxel_tensors)
labels, hdbscan_clusterer = find_maximum_clusters_hdbscan(features)
create_cluster_folders(output_folder, labels, stl_files, folder_path)

# Later in your script:
# labels, hdbscan_clusterer = find_optimal_hdbscan_clustering(features, 3)
# create_cluster_folders(output_folder, labels, stl_files, folder_path)


Total clusters formed (excluding noise): 251
Cluster sizes: {-1: 112, 0: 4, 1: 2, 2: 3, 3: 2, 4: 2, 5: 2, 6: 2, 7: 4, 8: 2, 9: 5, 10: 4, 11: 2, 12: 2, 13: 3, 14: 4, 15: 2, 16: 2, 17: 5, 18: 4, 19: 4, 20: 3, 21: 2, 22: 4, 23: 2, 24: 3, 25: 2, 26: 3, 27: 3, 28: 2, 29: 2, 30: 2, 31: 2, 32: 2, 33: 2, 34: 2, 35: 2, 36: 3, 37: 4, 38: 2, 39: 3, 40: 2, 41: 2, 42: 2, 43: 2, 44: 6, 45: 4, 46: 2, 47: 3, 48: 2, 49: 3, 50: 5, 51: 2, 52: 3, 53: 2, 54: 2, 55: 8, 56: 4, 57: 2, 58: 2, 59: 3, 60: 3, 61: 3, 62: 3, 63: 4, 64: 2, 65: 4, 66: 2, 67: 3, 68: 2, 69: 3, 70: 3, 71: 2, 72: 2, 73: 3, 74: 3, 75: 5, 76: 4, 77: 2, 78: 2, 79: 2, 80: 2, 81: 4, 82: 3, 83: 6, 84: 2, 85: 4, 86: 5, 87: 3, 88: 2, 89: 6, 90: 2, 91: 5, 92: 2, 93: 3, 94: 3, 95: 3, 96: 2, 97: 3, 98: 2, 99: 3, 100: 6, 101: 2, 102: 2, 103: 3, 104: 2, 105: 2, 106: 4, 107: 3, 108: 2, 109: 2, 110: 2, 111: 2, 112: 4, 113: 2, 114: 3, 115: 2, 116: 3, 117: 3, 118: 2, 119: 4, 120: 2, 121: 2, 122: 2, 123: 3, 124: 2, 125: 2, 126: 5, 127: 2, 128: 3, 129: 2, 

In [ ]:
# import numpy as np
# import os
# import numpy as np
# from stl import mesh
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler
# from sklearn.mixture import GaussianMixture
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import normalize
# from sklearn.mixture import GaussianMixture
# import networkx as nx
# from scipy.sparse.csgraph import connected_components
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.decomposition import PCA
# from sklearn.metrics import silhouette_score
# from stl import mesh
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense
# from tensorflow.keras.models import Model
# from sklearn.model_selection import train_test_split
# import os
# import shutil

# # Define a custom transformer to extract features using autoencoder
# class AutoencoderTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self, input_dim, encoding_dim, epochs=2, batch_size=32):
#         self.input_dim = input_dim
#         self.encoding_dim = encoding_dim
#         self.epochs = epochs
#         self.batch_size = batch_size
#         self.autoencoder = None
#         self.encoder = None

#     def fit(self, X, y=None):
#         input_layer = Input(shape=(self.input_dim,))
#         encoded = Dense(self.encoding_dim, activation='relu')(input_layer)
#         decoded = Dense(self.input_dim, activation='sigmoid')(encoded)

#         self.autoencoder = Model(input_layer, decoded)
#         self.encoder = Model(input_layer, encoded)

#         self.autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
#         X_train, X_valid = train_test_split(X, test_size=0.2, random_state=42)

#         self.autoencoder.fit(X_train, X_train,
#                              epochs=self.epochs,
#                              batch_size=self.batch_size,
#                              shuffle=True,
#                              validation_data=(X_valid, X_valid))
#         return self

#     def transform(self, X):
#         encoded_X = self.encoder.predict(X)
#         return encoded_X

# # Function to read STL file and extract vertices
# def read_stl(file_path):
#     mesh_data = mesh.Mesh.from_file(file_path)
#     vertices = mesh_data.vectors.reshape(-1, 9)
#     return vertices

# # Function to load STL files from a directory
# def load_stl_files(directory):
#     file_list = [f for f in os.listdir(directory) if f.endswith('.stl')]
#     stl_data = []
#     for file in file_list:
#         stl_data.append(read_stl(os.path.join(directory, file)))
#     return np.vstack(stl_data)

# # Define parameters
# stl_directory = '/content/PDSVision_dataset/PDSVision_dataset/stl_1000'
# encoding_dim = 32  # Number of neurons in the bottleneck layer of the autoencoder

# # Load STL files and extract features
# stl_data = load_stl_files(stl_directory)
# autoencoder_transformer = AutoencoderTransformer(input_dim=stl_data.shape[1], encoding_dim=encoding_dim)
# features = autoencoder_transformer.fit_transform(stl_data)
# print(features)

# # Normalize features
# scaler = StandardScaler()
# features_scaled = scaler.fit_transform(features)

# # Perform clustering using Gaussian Mixture Models
# gmm = GaussianMixture(n_components=1)  # You can adjust the number of clusters
# gmm.fit(features_scaled)
# cluster_labels = gmm.predict(features_scaled)
# print(cluster_labels)
# output_directory = '/content/drive/MyDrive/Datasets/encoded_clusters'

# # Create a directory to store the encoded clusters if it does not exist
# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)

# # Function to create folders for clusters and move STL files
# def organize_clusters(stl_directory, cluster_labels, max_files_per_cluster=5):
#     for cluster_id in np.unique(cluster_labels):
#         cluster_folder = os.path.join(output_directory, f'cluster_{cluster_id}')
#         os.makedirs(cluster_folder, exist_ok=True)

#     cluster_counts = {cluster_id: 0 for cluster_id in np.unique(cluster_labels)}
#     for file_name, cluster_id in zip(os.listdir(stl_directory), cluster_labels):
#         src_file_path = os.path.join(stl_directory, file_name)
#         dest_folder = os.path.join(output_directory, f'cluster_{cluster_id}')
#         dest_file_path = os.path.join(dest_folder, file_name)
#         if cluster_counts[cluster_id] < max_files_per_cluster:
#             shutil.copy(src_file_path, dest_file_path)
#             cluster_counts[cluster_id] += 1

# # Call the function to organize clusters and move STL files
# organize_clusters(stl_directory, cluster_labels, max_files_per_cluster=5)
# # Evaluate clustering using silhouette score
# # silhouette_avg = silhouette_score(features_scaled, cluster_labels)
# # print(f"Silhouette Score: {silhouette_avg}")

# # Optionally, you can perform dimensionality reduction for visualization
# # pca = PCA(n_components=2)
# # reduced_features = pca.fit_transform(features_scaled)

# # Now you can use the cluster labels for further analysis or visualization


Epoch 1/2
35902/35902 [==============================] - 92s 3ms/step - loss: -3034057.7500 - val_loss: -8596293.0000
Epoch 2/2
44877/44877 [==============================] - 74s 2ms/step
[[   0.         0.       916.56335 ... 1187.3474     0.      1265.87   ]
 [   0.         0.      1005.2433  ... 1271.3643     0.      1068.8318 ]
 [   0.         0.      1386.7888  ... 1660.4004     0.      1158.2697 ]
 ...
 [   0.         0.      3794.0986  ... 2843.0986     0.         0.     ]
 [   0.         0.      3407.3271  ... 1989.8209     0.         0.     ]
 [   0.         0.      3406.6575  ... 1980.7108     0.         0.     ]]
[0 0 0 ... 0 0 0]


In [ ]:
!pip install torch torchvision
!pip install torch-points3d


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.6/349.6 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 42.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torch-points3d to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.1/343.1 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━

In [ ]:
# 5
import trimesh
import numpy as np
import os


def load_stl_and_sample_points(stl_path, num_points=1024):
    mesh = trimesh.load(stl_path, force='mesh')
    if len(mesh.faces) == 0:
        raise ValueError("Loaded geometry does not contain faces.")
    points = mesh.sample(num_points)
    return points
# output = Returns a NumPy array of sampled points.
# This function is useful for extracting point cloud representations from 3D mesh data, which can be utilized in various 3D processing and machine learning tasks.

In [ ]:
# from torch_points3d.applications.pointnet2 import PointNet2
# import torch

# # Initialize PointNet model
# model = PointNet2(architecture="unet", input_nc=3, num_layers=4, output_nc=128, task='segmentation')
# model.eval()

# def process_point_clouds(point_clouds):
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     features_list = []
#     for points in point_clouds:
#         # Prepare tensor
#         points_tensor = torch.tensor(points).to(device).float().unsqueeze(0)
#         # Normalize points (assuming the model requires normalization)
#         center = points_tensor.mean(dim=1, keepdim=True)
#         scale = torch.norm(points_tensor - center, dim=-1, p=float('inf')).max(dim=1, keepdim=True)[0]
#         points_normalized = (points_tensor - center) / scale
#         # Extract features
#         with torch.no_grad():
#             features = model(points_normalized)
#         features_list.append(features.cpu().numpy())
#     return features_list

# # Example usage
# directory_path = '/content/drive/MyDrive/Datasets/encoded_clusters/toy_dataset'
# point_clouds = [load_stl_and_sample_points(os.path.join(directory_path, f)) for f in os.listdir(directory_path) if f.endswith('.stl')]
# features = process_point_clouds(point_clouds)


ModuleNotFoundError: No module named 'torch_points3d'

In [ ]:
# 6
import torch
import torch.nn as nn
import torch.nn.functional as F

class TNet(nn.Module):
    def __init__(self, k=3):
        super(TNet, self).__init__()
        self.k = k
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.fc3.bias.data.fill_(0)
        self.fc3.weight.data.uniform_(-0.0001, 0.0001)

    def forward(self, input):
        x = F.relu(self.bn1(self.conv1(input)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)
        iden = torch.eye(self.k).view(1, self.k*self.k).repeat(x.size(0), 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class PointNetFeat(nn.Module):
    def __init__(self):
        super(PointNetFeat, self).__init__()
        self.tnet = TNet(k=3)
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.tnet(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        return x

# Example instantiation and use of the network
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = PointNetFeat().to(device)
model.eval()


PointNetFeat(
  (tnet): TNet(
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (fc1): Linear(in_features=1024, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=256, bias=True)
    (fc3): Linear(in_features=256, out_features=9, bias=True)
    (relu): ReLU()
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  

In [ ]:
# 7
def normalize_point_cloud(points):
    centroid = np.mean(points, axis=0)
    points -= centroid
    furthest_distance = np.max(np.sqrt(np.sum(points ** 2, axis=1)))
    points /= furthest_distance
    return points
# output:Returns a NumPy array of normalized points with a mean distance of 1 from the origin.
# This approach ensures that the point cloud is centered and scaled in a way that each point's average distance to the origin is normalized to 1, which can be beneficial for certain machine learning algorithms and 3D data processing tasks.
# Subtracting the centroid from all points in a point cloud translates the entire point cloud so that its centroid (average position) aligns with the origin of the coordinate system. This centering process is a crucial preprocessing step for many 3D data processing tasks.









In [ ]:
# 8 repeat
# Assume `points` is a [N, 3] numpy array from an STL file
import trimesh
import numpy as np

def load_stl_and_sample_points(stl_path, num_points=1024):
    mesh = trimesh.load(stl_path, force='mesh')
    if len(mesh.faces) == 0:
        raise ValueError("Loaded geometry does not contain faces.")
    points = mesh.sample(num_points)
    return points


In [ ]:
# repeat 
def normalize_point_cloud(points):
    centroid = np.mean(points, axis=0)
    points -= centroid
    furthest_distance = np.max(np.sqrt(np.sum(points ** 2, axis=1)))
    points /= furthest_distance
    return points


In [ ]:
# 8
import torch

# Assume we have a `PointNetFeat` class defined as in the previous example
model = PointNetFeat().to(device)
model.eval()

def extract_features_from_point_cloud(file_path):
    points = load_stl_and_sample_points(file_path)
    points = normalize_point_cloud(points)
    points_tensor = torch.tensor(points, dtype=torch.float).to(device).unsqueeze(0).transpose(2, 1)  # Shape [1, 3, N]

    with torch.no_grad():
        features = model(points_tensor)
        return features

# Assuming an example STL file path
stl_file_path = '/content/drive/MyDrive/Datasets/encoded_clusters/cluster_0/122405_313-654d_flat1_prt.stl'
features = extract_features_from_point_cloud(stl_file_path)
print(features)  # Prints the shape of the extracted feature tensor


tensor([[ 0.0636,  0.0809, -0.0088,  ...,  0.1430,  0.0894,  0.1173]])


In [ ]:
# 9
import os
import torch
import trimesh
import numpy as np

# Assuming the PointNetFeat class and other dependencies are defined/imported as previously discussed.

def load_stl_and_sample_points(stl_path, num_points=1024):
    mesh = trimesh.load(stl_path, force='mesh')
    if len(mesh.faces) == 0:
        raise ValueError("Loaded geometry does not contain faces.")
    points = mesh.sample(num_points)
    return points

def normalize_point_cloud(points):
    centroid = np.mean(points, axis=0)
    points -= centroid
    furthest_distance = np.max(np.sqrt(np.sum(points ** 2, axis=1)))
    points /= furthest_distance
    return points

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = PointNetFeat().to(device)
model.eval()

def extract_features_from_point_cloud(file_path):
    points = load_stl_and_sample_points(file_path)
    points = normalize_point_cloud(points)
    points_tensor = torch.tensor(points, dtype=torch.float).to(device).unsqueeze(0).transpose(2, 1)  # Shape [1, 3, N]
    with torch.no_grad():
        features = model(points_tensor)
        return features.cpu().numpy()  # Return numpy array for easier handling outside PyTorch

# Directory containing STL files
directory_path = '/content/PDSVision_dataset/PDSVision_dataset/stl_1000'

# Process each STL file in the directory
for filename in os.listdir(directory_path):
    if filename.lower().endswith('.stl'):  # Check if the file is an STL file
        file_path = os.path.join(directory_path, filename)
        try:
            features = extract_features_from_point_cloud(file_path)
            print(f"Features for {filename}: {features.shape}")
            print(features)  # Print the numpy array of features
        except Exception as e:
            print(f"Failed to process {filename}: {e}")


Features for 138322_312-739d_flat1_prt.stl: (1, 1024)
[[-0.07723328  0.05101315 -0.03815539 ...  0.17216764  0.15107971
   0.13136259]]
Features for 135829_315-897d_flat1_prt.stl: (1, 1024)
[[-0.07662917  0.05651911 -0.03822494 ...  0.18979968  0.15128608
   0.13207352]]
Features for 138358_313-426d_flat1_prt.stl: (1, 1024)
[[-0.08007704  0.04680588 -0.03908504 ...  0.16827542  0.14879167
   0.12752397]]
Features for 130628_311-861d_flat1_prt.stl: (1, 1024)
[[-0.07999361  0.08497961 -0.03932609 ...  0.21475732  0.1358469
   0.15016618]]
Features for 134179_326-981d_prt.stl: (1, 1024)
[[-0.07633427  0.10008538 -0.03814109 ...  0.17265266  0.1120014
   0.13719559]]
Features for 135195_311-547d_flat1_prt.stl: (1, 1024)
[[-0.07921183  0.0999684  -0.04119218 ...  0.18976024  0.12214392
   0.14307217]]
Features for 138527_316-493d_flat1_prt.stl: (1, 1024)
[[-0.08027299  0.10318387 -0.0240179  ...  0.14675815  0.1488927
   0.19299586]]
Features for 132862_313-051d_flat1_prt.stl: (1, 1024)
[[-

In [ ]:
# 10
import os
import torch
import trimesh
import numpy as np
import hdbscan
import shutil

# Assuming the PointNetFeat class and other dependencies are defined/imported
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = PointNetFeat().to(device)
model.eval()

input_directory_path = '/content/PDSVision_dataset/PDSVision_dataset/stl_1000'
output_base_directory = '/content/drive/MyDrive/Datasets/hdbscan_clusters'


In [ ]:
all_features = []
filenames = []

for filename in os.listdir(input_directory_path):
    if filename.lower().endswith('.stl'):
        file_path = os.path.join(input_directory_path, filename)
        try:
            features = extract_features_from_point_cloud(file_path)
            all_features.append(features.flatten())
            filenames.append(filename)
            print(f"Processed {filename}")
        except Exception as e:
            print(f"Failed to process {filename}: {e}")

features_matrix = np.array(all_features)


Processed 138322_312-739d_flat1_prt.stl
Processed 135829_315-897d_flat1_prt.stl
Processed 138358_313-426d_flat1_prt.stl
Processed 130628_311-861d_flat1_prt.stl
Processed 134179_326-981d_prt.stl
Processed 135195_311-547d_flat1_prt.stl
Processed 138527_316-493d_flat1_prt.stl
Processed 132862_313-051d_flat1_prt.stl
Processed 137857_310-543d_flat1_prt.stl
Processed 130627_311-864d_flat1prt_prt.stl
Processed 138313_311-926d_flat1_prt.stl
Processed 143334_311-158d_flat1_prt.stl
Processed 135134_313-376d_flat1_prt.stl
Processed 142563_313-439d_flat1_prt.stl
Processed 143109_316-206d_flat1_prt.stl
Processed 143229_314-454d_flat_prt.stl
Processed 136050_313-423d_flat1_prt.stl
Processed 123818_311-076d_flat1_prt.stl
Processed 135129_311-734d_flat1_prt.stl
Processed 143363_313-701d_flat_prt.stl
Processed 134035_311-470d_flat1_prt.stl
Processed 123627_310-531d_flat1_prt.stl
Processed 138420_312-981d_flat1_prt.stl
Processed 131587_311-157d_flat1_prt.stl
Processed 135686_310-562d_flat1_prt.stl
Proce

In [ ]:
# Initialize HDBSCAN with parameters tuned for more clusters
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, gen_min_span_tree=True)
cluster_labels = clusterer.fit_predict(features_matrix)

if not os.path.exists(output_base_directory):
    os.makedirs(output_base_directory)

# Create a directory for each cluster label
cluster_directories = {}
for cluster_label in set(cluster_labels):
    if cluster_label != -1:  # Exclude noise if needed
        cluster_dir = os.path.join(output_base_directory, f'Cluster_{cluster_label}')
        os.makedirs(cluster_dir, exist_ok=True)
        cluster_directories[cluster_label] = cluster_dir


In [ ]:
for filename, cluster_label in zip(filenames, cluster_labels):
    if cluster_label != -1:
        src_path = os.path.join(input_directory_path, filename)
        dest_path = os.path.join(cluster_directories[cluster_label], filename)
        shutil.copy(src_path, dest_path)

print("Files have been copied to their respective cluster directories.")


Files have been copied to their respective cluster directories.


In [ ]:
# 12 views point net
import torch
import torch.nn as nn
import torch.nn.functional as F

class TNet(nn.Module):
    def __init__(self, k=3):
        super(TNet, self).__init__()
        self.k = k
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.fc3.bias.data.fill_(0)
        self.fc3.weight.data.uniform_(-0.0001, 0.0001)

    def forward(self, input):
        x = F.relu(self.bn1(self.conv1(input)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)
        iden = torch.eye(self.k).view(1, self.k*self.k).repeat(x.size(0), 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class PointNetFeat(nn.Module):
    def __init__(self):
        super(PointNetFeat, self).__init__()
        self.tnet = TNet(k=3)
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.tnet(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        return x

# Example instantiation and use of the network
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = PointNetFeat().to(device)
model.eval()


PointNetFeat(
  (tnet): TNet(
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (fc1): Linear(in_features=1024, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=256, bias=True)
    (fc3): Linear(in_features=256, out_features=9, bias=True)
    (relu): ReLU()
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  

In [ ]:
!pip install gudhi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.1 MB/s eta 0:00:00


In [ ]:

!pip install git+https://github.com/ranahanocka/MeshCNN.git


  Cloning https://github.com/ranahanocka/MeshCNN.git to /tmp/pip-req-build-7fgyiv86
  Running command git clone --filter=blob:none --quiet https://github.com/ranahanocka/MeshCNN.git /tmp/pip-req-build-7fgyiv86
  Resolved https://github.com/ranahanocka/MeshCNN.git to commit 5bf0b899d48eb204b9b73bc1af381be20f4d7df1
ERROR: git+https://github.com/ranahanocka/MeshCNN.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PointNetSetAbstraction(nn.Module):
    def __init__(self, num_points, radius, num_samples, in_channels, out_channels):
        super(PointNetSetAbstraction, self).__init__()
        self.num_points = num_points
        self.radius = radius
        self.num_samples = num_samples
        self.conv1 = nn.Conv1d(in_channels, out_channels, 1)
        self.conv2 = nn.Conv1d(out_channels, out_channels, 1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.max_pool = nn.MaxPool1d(num_samples)

    def forward(self, x):
        # Dummy implementation of sampling and grouping
        # You will need a proper way to perform farthest point sampling and ball query
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.max_pool(x)
        return x

class PointNetPlusPlus(nn.Module):
    def __init__(self):
        super(PointNetPlusPlus, self).__init__()
        self.sa1 = PointNetSetAbstraction(num_points=512, radius=0.2, num_samples=32, in_channels=3, out_channels=64)
        self.sa2 = PointNetSetAbstraction(num_points=128, radius=0.4, num_samples=64, in_channels=64, out_channels=128)
        self.fc1 = nn.Linear(128, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)  # 10 classes
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.sa1(x)
        x = self.sa2(x)
        x = x.view(batch_size, 128, -1).max(2)[0]

        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = PointNetPlusPlus().to(device)
model.eval()



PointNetPlusPlus(
  (sa1): PointNetSetAbstraction(
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (max_pool): MaxPool1d(kernel_size=32, stride=32, padding=0, dilation=1, ceil_mode=False)
  )
  (sa2): PointNetSetAbstraction(
    (conv1): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
    (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (max_pool): MaxPool1d(kernel_size=64, stride=64, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=128, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=

In [ ]:
import os
import torch
import trimesh
import numpy as np
import hdbscan
import shutil
from scipy.spatial.transform import Rotation as R
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil
import numpy as np
import trimesh
from gudhi import SimplexTree
import gudhi as gd
# Assuming the PointNetFeat class and other dependencies are defined/imported
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = PointNetFeat().to(device)
model.eval()

def load_stl_and_sample_points(stl_path, num_points=2048):
    """Loads an STL file, samples points from its surface, and returns them."""
    mesh = trimesh.load(stl_path, force='mesh')
    if len(mesh.faces) == 0:
        raise ValueError("Loaded geometry does not contain faces.")
    return mesh.sample(num_points)
def calculate_centroid(mesh_data):
    """Calculate the centroid of a mesh."""
    centroid = np.mean(mesh_data.vertices, axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    """Extract holes and cavities based on the topology of the mesh using a graph approach."""
    G = nx.Graph()
    for facet in mesh_data.faces:
        for i in range(3):
            edge = tuple(mesh_data.vertices[facet[i]])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(mesh_data.vertices[facet[next_i]])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(genus)

    return handles


def generate_views(points, num_views=12):
    """Generates multiple views by rotating the point cloud."""
    views = []
    rotations = np.linspace(0, 360, num_views, endpoint=False)
    for angle in rotations:
        rotation = R.from_euler('z', angle, degrees=True)
        rotated_points = rotation.apply(points)
        views.append(rotated_points)
    return views

def normalize_point_cloud(points):
    """Normalizes the point cloud to the unit sphere."""
    centroid = np.mean(points, axis=0)
    points -= centroid
    furthest_distance = np.max(np.sqrt(np.sum(points ** 2, axis=1)))
    points /= furthest_distance
    return points
def extract_topological_features(mesh):
    """Extract topological features from a mesh using GUDHI's SimplexTree."""
    simplex_tree = gd.SimplexTree()  # Initialize the simplex tree

    # Adding vertices (optional if you are only interested in edges)
    for index, vertex in enumerate(mesh.vertices):
        simplex_tree.insert([index], filtration=0)

    # Adding edges derived from the faces of the mesh
    for face in mesh.faces:
        for i in range(len(face)):
            simplex_tree.insert([face[i], face[(i+1) % len(face)]], filtration=0)

    # Compute persistence
    simplex_tree.persistence()

    # Get Betti numbers (Betti-0: connected components, Betti-1: holes)
    betti_numbers = simplex_tree.betti_numbers()
    num_holes = betti_numbers[1] if len(betti_numbers) > 1 else 0  # Assuming Betti-1 corresponds to holes

    return np.array([num_holes], dtype=np.float32)
  # Betti-1 corresponds to the number of holes

def extract_features_from_view(points):
    """Extracts features from a single view using the PointNet model."""
    points_tensor = torch.tensor(points, dtype=torch.float).to(device).unsqueeze(0).transpose(2, 1)  # Shape [1, 3, N]
    with torch.no_grad():
        features = model(points_tensor)
        return features.cpu().numpy()  # Return numpy array for easier handling outside PyTorch

input_directory_path = '/content/PDSVision_dataset/PDSVision_dataset/stl_1000'
output_base_directory = '/content/drive/MyDrive/Datasets/hdbscan_clusters_2'
all_features = []
filenames = []

for filename in os.listdir(input_directory_path):
    if filename.lower().endswith('.stl'):
        file_path = os.path.join(input_directory_path, filename)
        try:
            mesh = trimesh.load(file_path, force='mesh')
            points = load_stl_and_sample_points(file_path)
            points = normalize_point_cloud(points)
            views = generate_views(points)
            features_per_view = [extract_features_from_view(view) for view in views]
            surface = Surface()
            surface.load(file_path)
            surface_area = surface.get_area()
            bbv = surface.get_bounding_box_volume()
            combined_features = np.concatenate(features_per_view*8, axis=1).flatten()
            centroid = calculate_centroid(mesh)
            betti=extract_topological_features(mesh)

            holes = np.array([extract_holes_and_cavities(mesh)], dtype=np.float32)
            # combined_features = np.append(combined_features, centroid)
            combined_features = np.append(combined_features, betti)
            # combined_features = np.append(combined_features, surface_area)
            # combined_features = np.append(combined_features, bbv)
            all_features.append(combined_features)
            filenames.append(filename)
            print(f"Processed {filename}")
        except Exception as e:
            print(f"Failed to process {filename}: {e}")

features_matrix = np.array(all_features)
# Initialize HDBSCAN with parameters tuned for more clusters
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, gen_min_span_tree=True)
cluster_labels = clusterer.fit_predict(features_matrix)

if not os.path.exists(output_base_directory):
    os.makedirs(output_base_directory)

# Create a directory for each cluster label
cluster_directories = {}
for cluster_label in set(cluster_labels):
    if cluster_label != -1:  # Exclude noise if needed
        cluster_dir = os.path.join(output_base_directory, f'Cluster_{cluster_label}')
        os.makedirs(cluster_dir, exist_ok=True)
        cluster_directories[cluster_label] = cluster_dir
for filename, cluster_label in zip(filenames, cluster_labels):
    if cluster_label != -1:
        src_path = os.path.join(input_directory_path, filename)
        dest_path = os.path.join(cluster_directories[cluster_label], filename)
        shutil.copy(src_path, dest_path)

print("Files have been copied to their respective cluster directories.")


NameError: name 'PointNetFeat' is not defined

In [ ]:
import os
import torch
import trimesh
import numpy as np
import hdbscan
import shutil
from scipy.spatial.transform import Rotation as R
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil
import numpy as np
import trimesh
from gudhi import SimplexTree
import gudhi as gd
# Assuming the PointNetFeat class and other dependencies are defined/imported
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = PointNetPlusPlus().to(device)
model.eval()

def load_stl_and_sample_points(stl_path, num_points=2048):
    """Loads an STL file, samples points from its surface, and returns them."""
    mesh = trimesh.load(stl_path, force='mesh')
    if len(mesh.faces) == 0:
        raise ValueError("Loaded geometry does not contain faces.")
    return mesh.sample(num_points)
def calculate_centroid(mesh_data):
    """Calculate the centroid of a mesh."""
    centroid = np.mean(mesh_data.vertices, axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    """Extract holes and cavities based on the topology of the mesh using a graph approach."""
    G = nx.Graph()
    for facet in mesh_data.faces:
        for i in range(3):
            edge = tuple(mesh_data.vertices[facet[i]])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(mesh_data.vertices[facet[next_i]])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(genus)

    return handles


def generate_views(points, num_views=12):
    """Generates multiple views by rotating the point cloud."""
    views = []
    rotations = np.linspace(0, 360, num_views, endpoint=False)
    for angle in rotations:
        rotation = R.from_euler('z', angle, degrees=True)
        rotated_points = rotation.apply(points)
        views.append(rotated_points)
    return views

def normalize_point_cloud(points):
    """Normalizes the point cloud to the unit sphere."""
    centroid = np.mean(points, axis=0)
    points -= centroid
    furthest_distance = np.max(np.sqrt(np.sum(points ** 2, axis=1)))
    points /= furthest_distance
    return points
def extract_topological_features(mesh):
    """Extract topological features from a mesh using GUDHI's SimplexTree."""
    simplex_tree = gd.SimplexTree()  # Initialize the simplex tree

    # Adding vertices (optional if you are only interested in edges)
    for index, vertex in enumerate(mesh.vertices):
        simplex_tree.insert([index], filtration=0)

    # Adding edges derived from the faces of the mesh
    for face in mesh.faces:
        for i in range(len(face)):
            simplex_tree.insert([face[i], face[(i+1) % len(face)]], filtration=0)

    # Compute persistence
    simplex_tree.persistence()

    # Get Betti numbers (Betti-0: connected components, Betti-1: holes)
    betti_numbers = simplex_tree.betti_numbers()
    num_holes = betti_numbers[1] if len(betti_numbers) > 1 else 0  # Assuming Betti-1 corresponds to holes

    return np.array([num_holes], dtype=np.float32)
  # Betti-1 corresponds to the number of holes

def extract_features_from_view(points):
    """Extracts features from a single view using the PointNet model."""
    points_tensor = torch.tensor(points, dtype=torch.float).to(device).unsqueeze(0).transpose(2, 1)  # Shape [1, 3, N]
    with torch.no_grad():
        features = model(points_tensor)
        return features.cpu().numpy()  # Return numpy array for easier handling outside PyTorch

input_directory_path = '/content/PDSVision_dataset/PDSVision_dataset/stl_1000'
output_base_directory = '/content/drive/MyDrive/Datasets/hdbscan_clusters_5'
all_features = []
filenames = []

for filename in os.listdir(input_directory_path):
    if filename.lower().endswith('.stl'):
        file_path = os.path.join(input_directory_path, filename)
        try:
            mesh = trimesh.load(file_path, force='mesh')
            points = load_stl_and_sample_points(file_path)
            points = normalize_point_cloud(points)
            views = generate_views(points)
            features_per_view = [extract_features_from_view(view) for view in views]
            # surface = Surface()
            # surface.load(file_path)
            # surface_area = surface.get_area()
            # bbv = surface.get_bounding_box_volume()
            combined_features = np.concatenate(features_per_view*8, axis=1).flatten()
            centroid = calculate_centroid(mesh)
            betti=extract_topological_features(mesh)

            holes = np.array([extract_holes_and_cavities(mesh)], dtype=np.float32)
            # combined_features = np.append(combined_features, centroid)
            # combined_features = np.append(combined_features, betti)
            # combined_features = np.append(combined_features, surface_area)
            # combined_features = np.append(combined_features, bbv)
            all_features.append(combined_features)
            filenames.append(filename)
            print(f"Processed {filename}")
        except Exception as e:
            print(f"Failed to process {filename}: {e}")

features_matrix = np.array(all_features)
# Initialize HDBSCAN with parameters tuned for more clusters
clusterer = hdbscan.HDBSCAN(min_cluster_size=2,
                            min_samples=1, gen_min_span_tree=True)

cluster_labels = clusterer.fit_predict(features_matrix)

if not os.path.exists(output_base_directory):
    os.makedirs(output_base_directory)

# Create a directory for each cluster label
cluster_directories = {}
c=0
for cluster_label in set(cluster_labels):

    cluster_dir = os.path.join(output_base_directory, f'Cluster_{cluster_label}')
    os.makedirs(cluster_dir, exist_ok=True)
    cluster_directories[cluster_label] = cluster_dir





for filename, cluster_label in zip(filenames, cluster_labels):

    src_path = os.path.join(input_directory_path, filename)
    dest_path = os.path.join(cluster_directories[cluster_label], filename)
    shutil.copy(src_path, dest_path)


print("Files have been copied to their respective cluster directories.")


Processed 137373_311-813d_prt.stl
Processed 133142_312-779d_flat1_prt.stl
Processed 135017_315-723d_first_bend_oper_prt.stl
Processed 135188_311-847d_prt.stl
Processed 133888_313-912d_flat1_prt.stl
Processed 136089_311-878d_prt.stl
Processed 132804_315-382d_flat1_prt.stl
Processed 134476_308-375d_flat1_prt.stl
Processed 131614_313-988d_flat1_prt.stl
Processed 132805_313-968d_flat1_prt.stl
Processed 122562_312-909d_flat1_prt.stl
Processed 139558_834-110c_prt.stl
Processed 138725_310-268d_flat1_prt.stl
Processed 138188_313-729d_flat1_prt.stl
Processed 135197_311-575d_prt.stl
Processed 141476_310-484d_flat1_prt.stl
Processed 134236_311-080d_prt.stl
Processed 141959_311-676d_flat1_prt.stl
Processed 141886_313-441d_flat1_prt.stl
Processed 130339_311-960h_asm.stl
Processed 123356_313-088d_flat1_prt.stl
Processed 134453_313-393d_flat1_prt.stl
Processed 133577_311-737d_flat1_prt.stl
Processed 143366_313-063d_flat_wcurl_prt.stl
Processed 136222_311-384d_flat2_prt.stl
Processed 131196_308-386d_f

In [ ]:
import os
import numpy as np
import hdbscan
from stl import mesh
import networkx as nx

def calculate_holes(stl_path):
    """Calculate the number of holes in the STL file."""
    stl_mesh = mesh.Mesh.from_file(stl_path)
    G = nx.Graph()
    for data in stl_mesh.vectors:
        for i in range(3):
            edge = tuple(sorted([tuple(data[i]), tuple(data[(i + 1) % 3])]))
            if G.has_edge(*edge):
                G.remove_edge(*edge)
            else:
                G.add_edge(*edge)
    return nx.number_connected_components(G) - 1

def cluster_stl_files_in_directory(directory_path):
    """Cluster all STL files in a given directory based on the number of holes."""
    file_paths = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.stl')]
    if not file_paths:
        return

    features = np.array([calculate_holes(f) for f in file_paths])
    clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1)
    cluster_labels = clusterer.fit_predict(features.reshape(-1, 1))

    # Organize files by new clusters
    cluster_dirs = {}
    for file_path, label in zip(file_paths, cluster_labels):
        cluster_dir = os.path.join(directory_path, str(label))
        if label not in cluster_dirs:
            os.makedirs(cluster_dir, exist_ok=True)
            cluster_dirs[label] = cluster_dir
        os.rename(file_path, os.path.join(cluster_dir, os.path.basename(file_path)))

    print(f"Re-clustered STL files in {directory_path} into subdirectories for each new cluster.")

# Main processing of clusters
base_directory = '/content/drive/MyDrive/Datasets/hdbscan_clusters_5/'
cluster_directories = [os.path.join(base_directory, d) for d in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, d))]

for cluster_dir in cluster_directories:
    cluster_stl_files_in_directory(cluster_dir)


Re-clustered STL files in /content/drive/MyDrive/Datasets/hdbscan_clusters_5/Cluster_0 into subdirectories for each new cluster.
Re-clustered STL files in /content/drive/MyDrive/Datasets/hdbscan_clusters_5/Cluster_1 into subdirectories for each new cluster.
Re-clustered STL files in /content/drive/MyDrive/Datasets/hdbscan_clusters_5/Cluster_2 into subdirectories for each new cluster.
Re-clustered STL files in /content/drive/MyDrive/Datasets/hdbscan_clusters_5/Cluster_3 into subdirectories for each new cluster.
Re-clustered STL files in /content/drive/MyDrive/Datasets/hdbscan_clusters_5/Cluster_4 into subdirectories for each new cluster.
Re-clustered STL files in /content/drive/MyDrive/Datasets/hdbscan_clusters_5/Cluster_5 into subdirectories for each new cluster.
Re-clustered STL files in /content/drive/MyDrive/Datasets/hdbscan_clusters_5/Cluster_6 into subdirectories for each new cluster.
Re-clustered STL files in /content/drive/MyDrive/Datasets/hdbscan_clusters_5/Cluster_7 into subdi

In [ ]:
# import os
# import numpy as np
# from stl import mesh
# import networkx as nx
# from sklearn.cluster import KMeans

# def calculate_holes(stl_path):
#     """Calculate the number of holes in the STL file."""
#     stl_mesh = mesh.Mesh.from_file(stl_path)
#     G = nx.Graph()
#     for data in stl_mesh.vectors:
#         for i in range(3):
#             edge = tuple(sorted([tuple(data[i]), tuple(data[(i + 1) % 3])]))
#             if G.has_edge(*edge):
#                 G.remove_edge(*edge)
#             else:
#                 G.add_edge(*edge)
#     return nx.number_connected_components(G) - 1

# def cluster_stl_files_in_directory(directory_path, n_clusters=):
#     """Cluster all STL files in a given directory based on the number of holes using KMeans."""
#     file_paths = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.stl')]
#     if not file_paths:
#         return

#     features = np.array([calculate_holes(f) for f in file_paths])
#     kmeans = KMeans(n_clusters=n_clusters, random_state=0)
#     cluster_labels = kmeans.fit_predict(features.reshape(-1, 1))

#     # Organize files by new clusters
#     cluster_dirs = {}
#     for file_path, label in zip(file_paths, cluster_labels):
#         cluster_dir = os.path.join(directory_path, f"cluster_{label}")
#         if label not in cluster_dirs:
#             os.makedirs(cluster_dir, exist_ok=True)
#             cluster_dirs[label] = cluster_dir
#         os.rename(file_path, os.path.join(cluster_dir, os.path.basename(file_path)))

#     print(f"Re-clustered STL files in {directory_path} into subdirectories for each new cluster.")

# # Main processing of clusters
# base_directory = '/content/drive/MyDrive/Datasets/hdbscan_clusters_4/'
# cluster_directories = [os.path.join(base_directory, d) for d in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, d))]

# for cluster_dir in cluster_directories:
#     cluster_stl_files_in_directory(cluster_dir)


ValueError: n_samples=4 should be >= n_clusters=5.

In [ ]:
!pip install gudhi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.9 MB/s eta 0:00:00


In [ ]:
!pip install hdbscan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039283 sha256=400bc9b874ea9156c198eb4de314f44d24b41bc1b9517d014eb7b4a2b50f73eb
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
Successfully built hdbscan
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [ ]:
# import os
# import numpy as np
# from sklearn.preprocessing import StandardScaler
# import gudhi as gd
# import hdbscan
# import matplotlib.pyplot as plt
# from stl import mesh

# # Paths
# dataset_folder = "/content/PDSVision_dataset/PDSVision_dataset/stl_1000"
# output_folder = "/content/drive/MyDrive/Datasets/hdbscan_clusters_6/"

# # Ensure the output directory exists
# os.makedirs(output_folder, exist_ok=True)

# # Load data from STL files
# def load_data_from_stl(folder):
#     files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.stl')]
#     if not files:
#         raise ValueError("No STL files found in the specified folder.")
#     data_list = []
#     for file in files:
#         try:
#             stl_mesh = mesh.Mesh.from_file(file)
#             points = np.vstack((stl_mesh.v0, stl_mesh.v1, stl_mesh.v2))
#             data_list.append(points)
#         except Exception as e:
#             print(f"Error loading {file}: {e}")
#     if not data_list:
#         raise ValueError("No data could be loaded from the STL files.")
#     return np.vstack(data_list)

# try:
#     data = load_data_from_stl(dataset_folder)
# except ValueError as e:
#     print(e)
#     data = None

# if data is not None:
#     # Standardize data
#     scaler = StandardScaler()
#     data_scaled = scaler.fit_transform(data)

#     # Step 2: Construct a Simplicial Complex
#     rips_complex = gd.RipsComplex(points=data_scaled, max_edge_length=2.0)
#     simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

#     # Step 3: Compute Persistent Homology
#     persistence = simplex_tree.persistence()

#     # Extract 1-dimensional features (e.g., loops)
#     persistence_intervals = simplex_tree.persistence_intervals_in_dimension(1)

#     # Convert persistence intervals to a persistence landscape
#     landscape = gd.representations.Landscape(resolution=1000)
#     landscape_features = landscape.fit_transform([persistence_intervals])

#     # Reshape landscape features for clustering
#     landscape_features_reshaped = landscape_features.reshape(len(landscape_features), -1)

#     # Step 5: Cluster the data using HDBSCAN
#     clusterer = hdbscan.HDBSCAN(min_cluster_size=5)
#     clusters = clusterer.fit_predict(landscape_features_reshaped)

#     # Save the clustering results
#     output_file = os.path.join(output_folder, 'hdbscan_clusters.csv')
#     np.savetxt(output_file, clusters, delimiter=',')

#     # Optional: Visualize the clustering result
#     plt.scatter(data_scaled[:, 0], data_scaled[:, 1], c=clusters, cmap='rainbow')
#     plt.title('HDBSCAN Clustering Result')
#     plt.savefig(os.path.join(output_folder, 'hdbscan_clusters.png'))
#     plt.show()

#     print("Clustering results saved to:", output_folder)
# else:
#     print("Data loading failed. No clustering performed.")


In [ ]:
# import os
# import numpy as np
# from sklearn.preprocessing import StandardScaler
# import gudhi as gd
# import hdbscan
# import matplotlib.pyplot as plt
# from stl import mesh

# # Paths
# dataset_folder = "/content/PDSVision_dataset/PDSVision_dataset/stl_1000"
# output_folder = "/content/drive/MyDrive/Datasets/hdbscan_clusters_6/"

# # Ensure the output directory exists
# os.makedirs(output_folder, exist_ok=True)

# # Load data from STL files
# def load_data_from_stl(folder, max_points=10000):
#     files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.stl')]
#     if not files:
#         raise ValueError("No STL files found in the specified folder.")
#     data_list = []
#     for file in files:
#         try:
#             stl_mesh = mesh.Mesh.from_file(file)
#             points = np.vstack((stl_mesh.v0, stl_mesh.v1, stl_mesh.v2))
#             if len(points) > max_points:
#                 indices = np.random.choice(len(points), max_points, replace=False)
#                 points = points[indices]
#             data_list.append(points)
#         except Exception as e:
#             print(f"Error loading {file}: {e}")
#     if not data_list:
#         raise ValueError("No data could be loaded from the STL files.")
#     return np.vstack(data_list)

# try:
#     data = load_data_from_stl(dataset_folder)
# except ValueError as e:
#     print(e)
#     data = None

# if data is not None:
#     # Standardize data
#     scaler = StandardScaler()
#     data_scaled = scaler.fit_transform(data)

#     # Step 2: Construct a Simplicial Complex
#     max_edge_length = 1.0  # Adjust this value based on your data and memory constraints
#     rips_complex = gd.RipsComplex(points=data_scaled, max_edge_length=max_edge_length)
#     simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

#     # Step 3: Compute Persistent Homology
#     persistence = simplex_tree.persistence()

#     # Extract 1-dimensional features (e.g., loops)
#     persistence_intervals = simplex_tree.persistence_intervals_in_dimension(1)

#     # Convert persistence intervals to a persistence landscape
#     landscape = gd.representations.Landscape(resolution=1000)
#     landscape_features = landscape.fit_transform([persistence_intervals])

#     # Reshape landscape features for clustering
#     landscape_features_reshaped = landscape_features.reshape(len(landscape_features), -1)

#     # Step 5: Cluster the data using HDBSCAN
#     clusterer = hdbscan.HDBSCAN(min_cluster_size=5)
#     clusters = clusterer.fit_predict(landscape_features_reshaped)

#     # Save the clustering results
#     output_file = os.path.join(output_folder, 'hdbscan_clusters.csv')
#     np.savetxt(output_file, clusters, delimiter=',')

#     # Optional: Visualize the clustering result
#     plt.scatter(data_scaled[:, 0], data_scaled[:, 1], c=clusters, cmap='rainbow')
#     plt.title('HDBSCAN Clustering Result')
#     plt.savefig(os.path.join(output_folder, 'hdbscan_clusters.png'))
#     plt.show()

#     print("Clustering results saved to:", output_folder)
# else:
#     print("Data loading failed. No clustering performed.")


In [ ]:
# import os
# import numpy as np
# from sklearn.preprocessing import StandardScaler
# import gudhi as gd
# import hdbscan
# import matplotlib.pyplot as plt
# from stl import mesh

# # Paths
# dataset_folder = "/content/PDSVision_dataset/PDSVision_dataset/stl_1000"
# output_folder = "/content/drive/MyDrive/Datasets/hdbscan_clusters_6/"

# # Ensure the output directory exists
# os.makedirs(output_folder, exist_ok=True)

# # Load data from STL files in batches
# def load_data_from_stl(folder, max_files=10, max_points_per_file=10000):
#     files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.stl')]
#     if not files:
#         raise ValueError("No STL files found in the specified folder.")
#     data_list = []
#     for i, file in enumerate(files):
#         if i >= max_files:
#             break
#         try:
#             stl_mesh = mesh.Mesh.from_file(file)
#             points = np.vstack((stl_mesh.v0, stl_mesh.v1, stl_mesh.v2))
#             if len(points) > max_points_per_file:
#                 indices = np.random.choice(len(points), max_points_per_file, replace=False)
#                 points = points[indices]
#             data_list.append(points)
#         except Exception as e:
#             print(f"Error loading {file}: {e}")
#     if not data_list:
#         raise ValueError("No data could be loaded from the STL files.")
#     return np.vstack(data_list)

# # Process the STL files in batches
# batch_size = 10  # Number of files per batch
# all_clusters = []

# files = [os.path.join(dataset_folder, f) for f in os.listdir(dataset_folder) if f.endswith('.stl')]
# num_batches = (len(files) + batch_size - 1) // batch_size

# for batch_idx in range(num_batches):
#     batch_files = files[batch_idx * batch_size : (batch_idx + 1) * batch_size]
#     try:
#         data = load_data_from_stl(dataset_folder, max_files=batch_size)
#     except ValueError as e:
#         print(e)
#         continue

#     if data is not None:
#         # Standardize data
#         scaler = StandardScaler()
#         data_scaled = scaler.fit_transform(data)

#         # Construct a Simplicial Complex
#         rips_complex = gd.RipsComplex(points=data_scaled, max_edge_length=1.0)
#         simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

#         # Compute Persistent Homology
#         persistence = simplex_tree.persistence()

#         # Extract 1-dimensional features (e.g., loops)
#         persistence_intervals = simplex_tree.persistence_intervals_in_dimension(1)

#         # Convert persistence intervals to a persistence landscape
#         landscape = gd.representations.Landscape(resolution=1000)
#         landscape_features = landscape.fit_transform([persistence_intervals])

#         # Reshape landscape features for clustering
#         landscape_features_reshaped = landscape_features.reshape(len(landscape_features), -1)

#         # Cluster the data using HDBSCAN
#         clusterer = hdbscan.HDBSCAN(min_cluster_size=5)
#         clusters = clusterer.fit_predict(landscape_features_reshaped)

#         all_clusters.append(clusters)

#         # Optional: Visualize the clustering result
#         plt.scatter(data_scaled[:, 0], data_scaled[:, 1], c=clusters, cmap='rainbow')
#         plt.title(f'HDBSCAN Clustering Result for Batch {batch_idx + 1}')
#         plt.savefig(os.path.join(output_folder, f'hdbscan_clusters_batch_{batch_idx + 1}.png'))
#         plt.show()

# # Combine and save all clustering results
# all_clusters_combined = np.concatenate(all_clusters)
# output_file = os.path.join(output_folder, 'hdbscan_clusters.csv')
# np.savetxt(output_file, all_clusters_combined, delimiter=',')

# print("Clustering results saved to:", output_folder)


In [ ]:
# import os
# import numpy as np
# from sklearn.preprocessing import StandardScaler
# import gudhi as gd
# import hdbscan
# import matplotlib.pyplot as plt
# from stl import mesh

# # Paths
# dataset_folder = "/content/PDSVision_dataset/PDSVision_dataset/stl_1000"
# output_folder = "/content/drive/MyDrive/Datasets/hdbscan_clusters_6/"

# # Ensure the output directory exists
# os.makedirs(output_folder, exist_ok=True)

# # Load data from an STL file
# def load_data_from_stl(file, max_points=10000):
#     stl_mesh = mesh.Mesh.from_file(file)
#     points = np.vstack((stl_mesh.v0, stl_mesh.v1, stl_mesh.v2))
#     if len(points) > max_points:
#         indices = np.random.choice(len(points), max_points, replace=False)
#         points = points[indices]
#     return points

# # Process each STL file independently
# persistence_diagrams_list = []
# files = [os.path.join(dataset_folder, f) for f in os.listdir(dataset_folder) if f.endswith('.stl')]

# for file in files:
#     try:
#         data = load_data_from_stl(file)
#     except Exception as e:
#         print(f"Error loading {file}: {e}")
#         continue

#     # Standardize data
#     scaler = StandardScaler()
#     data_scaled = scaler.fit_transform(data)

#     # Construct a Simplicial Complex
#     rips_complex = gd.RipsComplex(points=data_scaled, max_edge_length=1.0)
#     simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

#     # Compute Persistent Homology
#     persistence = simplex_tree.persistence()

#     # Extract 1-dimensional persistence intervals
#     persistence_intervals = simplex_tree.persistence_intervals_in_dimension(1)
#     persistence_diagrams_list.append(persistence_intervals)

# # Check if there are any persistence intervals
# if not persistence_diagrams_list:
#     print("No persistence intervals found. No clustering performed.")
# else:
#     # Vectorize the persistence diagrams for clustering
#     def vectorize_persistence_diagrams(diagrams):
#         vectorized = []
#         for diag in diagrams:
#             if diag.size == 0:  # Check if the diagram is empty
#                 vectorized.append([0, 0])  # Add dummy data if empty
#             else:
#                 births, deaths = diag[:, 0], diag[:, 1]
#                 vectorized.append(np.hstack((births, deaths)))
#         return np.array(vectorized)

#     vectorized_diagrams = vectorize_persistence_diagrams(persistence_diagrams_list)

#     # Ensure the data is a 2D array
#     if vectorized_diagrams.ndim == 1:
#         vectorized_diagrams = vectorized_diagrams.reshape(1, -1)

#     # Step 5: Cluster the data using HDBSCAN
#     clusterer = hdbscan.HDBSCAN(min_cluster_size=5)
#     clusters = clusterer.fit_predict(vectorized_diagrams)

#     # Save the clustering results
#     output_file = os.path.join(output_folder, 'hdbscan_clusters.csv')
#     np.savetxt(output_file, clusters, delimiter=',')

#     # Optional: Visualize the clustering result (using PCA for 2D visualization)
#     from sklearn.decomposition import PCA
#     pca = PCA(n_components=2)
#     data_2d = pca.fit_transform(vectorized_diagrams)
#     plt.scatter(data_2d[:, 0], data_2d[:, 1], c=clusters, cmap='rainbow')
#     plt.title('HDBSCAN Clustering Result')
#     plt.savefig(os.path.join(output_folder, 'hdbscan_clusters.png'))
#     plt.show()

#     print("Clustering results saved to:", output_folder)
